In [ ]:
from yambopy.dbs import YamboLatticeDB, YamboElectronsDB
from yambopy import *
import matplotlib.pyplot as plt
from yambopy.lattice import car_red, red_car
import matplotlib.pylab as pylab
from matplotlib.ticker import MultipleLocator
import matplotlib.lines as  mlines
shared_dir = '/Users/6179304/Developer/calculations/WS2/'
# savedb_k = YamboLatticeDB.from_db_file(f'{shared_dir}nscf-6x6x1/out/ws2.save/SAVE')
lat_k = YamboLatticeDB.from_db_file(folder=f'{shared_dir}/nscf-30x30x1/out/ws2.save/SAVE', Expand=False)
from yambopy.wannier import *
hrk=HR(f'{shared_dir}/nscf-wannier-30x30x1/ws2')
hrq=HR(f'{shared_dir}/nscf-wannier-30x30x1/ws2')
nnkp_kgrid = symmetrized_mp_grid([30,30,1],lat_k)
nnkp_qgrid = symmetrized_mp_grid([15,15,1],lat_k)
nnkp_kgrid.generate()
nnkp_qgrid.generate()
# nnkp_kgrid.k

# hrk=HR(f'{shared_dir}/nscf-wannier-30x30x1/ws2')
# hrq=HR(f'{shared_dir}/nscf-wannier-30x30x1/ws2')
# nnkp_kgrid = tb_Monkhorst_Pack([30,30,1],lat_k)
# nnkp_qgrid = tb_Monkhorst_Pack([30,30,1],lat_k)
# nnkp_kgrid.generate()
# nnkp_qgrid.generate()

model = TBMODEL.from_wannier_files(
    hr_file=f'{shared_dir}nscf-wannier-30x30x1/ws2_hr.dat',
    wsvec_file=f'{shared_dir}nscf-wannier-30x30x1/ws2_wsvec.dat',
    #xyz_file=f'{WORK_PATH}/w90files/hBN_centres.xyz',
    win_file=f'{shared_dir}nscf-wannier-30x30x1/ws2.win'
)
model.set_mpgrid(nnkp_kgrid)


# supercell_model = model.supercell([2,2,1])
supercell_model = model.supercell([2,2,1])
supercell_model.hamilton(nnkp_kgrid.k).shape
supercell_model.to_hr_file(f'{shared_dir}nscf-wannier-30x30x1/ws2_hr_supercell_hr.dat')
hrk_supercell=HR(f'{shared_dir}nscf-wannier-30x30x1/ws2_hr_supercell')

supercell_model.lat = supercell_model.uc

supercell_model.set_mpgrid(nnkp_kgrid)

# Define path in reduced coordinates using Class Path
npoints = 30

path_kpoints = Path([[[  0.0,  0.0,  0.0],'$\Gamma$'],
                     [[0.5, 0.0,  0.0],'M'],  
              [[  1/3,  1/3,  0.0],'K'],
              [[  0.0,  0.0,  0.0],'$\Gamma$']],[npoints,npoints,npoints-1] )
klist = path_kpoints.get_klist()
# model.set_mpgrid(klist)
fermie = -4
supercell_model.solve_ham_from_hr(supercell_model, hrk_supercell, fermie=fermie)

hlm = supercell_model.get_hlm(supercell_model, hrk_supercell, from_hr=False)
# cpot = CoulombPotentials(v0=0.00,lc=8.0, w=0.0, r0=1.0,ngrid=[(91÷/900)**2*30,30,1], lattice=lat_k, ediel=[1.0,1.0,1.0],tolr=0.001)
cpot = CoulombPotentials(v0=0.00,lc=8.0, w=0.0, r0=1.0,ngrid=[30,30,1], lattice=lat_k, ediel=[1.0,1.0,1.0],tolr=0.001)
# cpot = CoulombPotentials(v0=0.40,lc=13.174613459947587, w=0.40, r0=31,ngrid=[16,16,1], lattice=lat_k, ediel=[1.0,3.00,1.0],tolr=0.001)
savedb_path = f'{shared_dir}/nscf-30x30x1/out/ws2.save/SAVE'
# kernel_path = f'{shared_dir}bse/out/ws2.save/ws2BSE/'
h2p = H2P(supercell_model, savedb_path, qmpgrid=nnkp_kgrid, kernel_path=None, method='model',ctype='v2dt2', cpot=cpot,bse_nv=2,bse_nc=2,run_parallel=True,ktype='NOIP',eta=0.010,gammaonly=False)


E_k_supercell= np.array(supercell_model.get_eigenval(klist[:,:3],from_hr=True))
E_k= np.array(model.get_eigenval(klist[:,:3],from_hr=True))

kdistance = path_kpoints.distances

path_kpoints.distances
from yambopy.plot.bandstructure import YambopyBandStructure
tb_ebands = YambopyBandStructure(E_k,klist,kpath=path_kpoints)
tb_kdist = tb_ebands.distances

from qepy.lattice import Path, calculate_distances 

fig, ax = plt.subplots()

kpoints_dists = calculate_distances(klist)
for ib in range(E_k_supercell.shape[1]):
    # ax.plot(kpoints_dists, E_k[:,ib], c='black', lw=2.0)
    ax.plot(tb_kdist, E_k_supercell[:,ib], c='red', ls = '--', lw=2.0)

for ib in range(E_k.shape[1]):
    # ax.plot(kpoints_dists, E_k[:,ib], c='black', lw=2.0)
    ax.plot(tb_kdist, E_k[:,ib], c='black', ls = '-.', lw=2.0)

legend_entries = [
    mlines.Line2D([], [], color='red', label=f'supercell'),
    mlines.Line2D([], [], color='black', label=f'unitcell'),

]

ax.set_ylim(-8,0)
ax.legend(handles=legend_entries)
model.set_mpgrid(nnkp_kgrid)

# Define path in reduced coordinates using Class Path
npoints = 30

path_kpoints = Path([[[  0.0,  0.0,  0.0],'$\Gamma$'],
                     [[0.5, 0.0,  0.0],'M'],  
              [[  1/3,  1/3,  0.0],'K'],
              [[  0.0,  0.0,  0.0],'$\Gamma$']],[npoints,npoints,npoints-1] )
klist = path_kpoints.get_klist()
# model.set_mpgrid(klist)
fermie = -4
model.solve_ham_from_hr(lat_k, hrk, fermie=fermie)

hlm = model.get_hlm(lat_k.lat, hrk)
# cpot = CoulombPotentials(v0=0.00,lc=8.0, w=0.0, r0=1.0,ngrid=[(91÷/900)**2*30,30,1], lattice=lat_k, ediel=[1.0,1.0,1.0],tolr=0.001)
cpot = CoulombPotentials(v0=0.00,lc=8.0, w=0.0, r0=1.0,ngrid=[6,6,1], lattice=lat_k, ediel=[1.0,1.0,1.0],tolr=0.001)
# cpot = CoulombPotentials(v0=0.40,lc=13.174613459947587, w=0.40, r0=31,ngrid=[16,16,1], lattice=lat_k, ediel=[1.0,3.00,1.0],tolr=0.001)
savedb_path = f'{shared_dir}/nscf-30x30x1/out/ws2.save/SAVE'
# kernel_path = f'{shared_dir}bse/out/ws2.save/ws2BSE/'
h2p = H2P(model, savedb_path, qmpgrid=nnkp_kgrid, kernel_path=None, method='model',ctype='v2dt2', cpot=cpot,bse_nv=2,bse_nc=2
,run_parallel=True,ktype='NOIP',eta=0.010,gammaonly=False)

